In [1]:
%load_ext autoreload
%autoreload 2

import math
import random
import tempfile
import tensorflow as tf
import time

import numpy as np

from collections import defaultdict

from tf_rl.controller import KerasDDPG
from tf_rl.models     import DDPGPolicyMLP, DDPGValueMLP
from tf_rl.simulation import SinglePendulum
from tf_rl            import simulate

from keras import backend as K

SINGLE_PENDULUM_PARAMS = {
    'g_ms2': 9.8, # acceleration due to gravity, in m/s^2
    'l1_m': 1.0, # length of pendulum 1 in m
    'm1_kg': 1.0, # mass of pendulum 1 in kg
    'damping': 0.2,
    'max_control_input': 6.0
}

Using TensorFlow backend.


ImportError: cannot import name SinglePendulum

## Load New Model

In [ ]:
actor = PolicyMLP(SinglePendulum.observation_size, [200, 200, 1], ['relu', 'relu', 'tanh'])
critic = ValueMLP(SinglePendulum.observation_size,SinglePendulum.action_size,[200, 200, 1],['relu', 'relu', 'linear'], regularizer=True)

In [ ]:
current_controller = KerasDDPG(SinglePendulum.observation_size, 
                               SinglePendulum.action_size, actor, critic, discount_rate=0.99, 
                               exploration_period=4000)

## Train Model

In [ ]:
fast_mode = False

if fast_mode:
    FPS, SPEED, RES = 5, 20.0, 0.03
else:
    FPS, SPEED, RES = 60, 1., 0.01

try:
    while True:
        d = SinglePendulum(SINGLE_PENDULUM_PARAMS)
        simulate(d, current_controller, fps=FPS,
                 simulation_resolution=RES,
                 action_every=3,
                 disable_training=False,
                 reset_every=300,
                 visualize=True)
except KeyboardInterrupt:
    print("Interrupted")

## Load Model Checkpoint and Evaluate with No Training or Exploration

In [ ]:
current_controller.restore_checkpoint('/home/mderry/local_data/rl_logs/pendulum_checkpoint_500001')

In [ ]:
fast_mode = False

if fast_mode:
    FPS, SPEED, RES = 5, 20.0, 0.03
else:
    FPS, SPEED, RES = 60, 1., 0.0001

SINGLE_PENDULUM_PARAMS = {
    'g_ms2': 9.8, # acceleration due to gravity, in m/s^2
    'l1_m': 1.0, # length of pendulum 1 in m
    'm1_kg': 1.0, # mass of pendulum 1 in kg
    'damping': 0.2,
    'max_control_input': 6.0
}
d = SinglePendulum(SINGLE_PENDULUM_PARAMS)
try:
    while True:
        simulate(d, current_controller, fps=FPS, simulation_resolution=RES, wait=False, action_every=3, disable_training=True, ignore_exploration=True)
except KeyboardInterrupt:
    print("Interrupted")